DataSet Info

Loading the Data

In [ ]:
import pandas as pd
df = pd.read_csv("E:\\Data Analysis\\Projects\\Data Analysis End to end\\Data\\Sample - Superstore.csv", encoding='latin-1')

In [ ]:
df.shape, df.head(3)

((9994, 21),
    Row ID        Order ID Order Date   Ship Date     Ship Mode Customer ID  \
 0       1  CA-2016-152156  11/8/2016  11/11/2016  Second Class    CG-12520   
 1       2  CA-2016-152156  11/8/2016  11/11/2016  Second Class    CG-12520   
 2       3  CA-2016-138688  6/12/2016   6/16/2016  Second Class    DV-13045   
 
      Customer Name    Segment        Country         City  ... Postal Code  \
 0      Claire Gute   Consumer  United States    Henderson  ...       42420   
 1      Claire Gute   Consumer  United States    Henderson  ...       42420   
 2  Darrin Van Huff  Corporate  United States  Los Angeles  ...       90036   
 
    Region       Product ID         Category Sub-Category  \
 0   South  FUR-BO-10001798        Furniture    Bookcases   
 1   South  FUR-CH-10000454        Furniture       Chairs   
 2    West  OFF-LA-10000240  Office Supplies       Labels   
 
                                         Product Name   Sales  Quantity  \
 0                  Bush Somer

In [ ]:
df.info()
df.columns

<class 'pandas.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   str    
 2   Order Date     9994 non-null   str    
 3   Ship Date      9994 non-null   str    
 4   Ship Mode      9994 non-null   str    
 5   Customer ID    9994 non-null   str    
 6   Customer Name  9994 non-null   str    
 7   Segment        9994 non-null   str    
 8   Country        9994 non-null   str    
 9   City           9994 non-null   str    
 10  State          9994 non-null   str    
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   str    
 13  Product ID     9994 non-null   str    
 14  Category       9994 non-null   str    
 15  Sub-Category   9994 non-null   str    
 16  Product Name   9994 non-null   str    
 17  Sales          9994 non-null   float64
 18  Quantity       9994

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='str')

Cleaning Summary

Standarizing column names

In [ ]:
df.columns = (df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_"))
df.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub_category',
       'product_name', 'sales', 'quantity', 'discount', 'profit'],
      dtype='str')

Converting data types to correct ones

In [ ]:
# When applied, it instructs the function to handle any values that cannot be successfully converted by forcing them into a "missing" or "null" data representation -- errors = "coerce"

df["order_date"] = pd.to_datetime(df["order_date"], errors = "coerce")
df["ship_date"] = pd.to_datetime(df["ship_date"], errors = "coerce")

num_cols = ["sales", "profit", "discount"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors = "coerce").astype("Float64")

df["quantity"] = pd.to_numeric(df["quantity"], errors = "coerce").astype("Int64")

Checking nulls and nas

In [ ]:
print(df[num_cols + ["quantity"]].isna().sum())
print(df[["order_date", "ship_date"]].isna().sum())

sales       0
profit      0
discount    0
quantity    0
dtype: int64
order_date    0
ship_date     0
dtype: int64


Checking duplicate rows

In [ ]:
df.duplicated().sum()
df["row_id"].duplicated().sum()

# if found duplicates then drop them using
# df = df.drop_duplicates()

np.int64(0)

Nulls in key columns

In [ ]:
key_cols = ["order_id", "order_date", "customer_id", "product_id"]
df[key_cols].isna().sum()

# if any of them is null then drop these rows 
# df = df.dropna(subset = key_cols)

order_id       0
order_date     0
customer_id    0
product_id     0
dtype: int64

Checking dates logic

In [ ]:
bad_ship = df[df["ship_date"] < df["order_date"]]
bad_ship.shape[0]

# If small count: remove or fix (usually remove)
# df = df[df["ship_date"] >= df["order_date"]]

0

Cleaning text columns

In [ ]:
text_cols = ["ship_mode", "customer_name", "segment", "country", "city", "state", "region", "product_name", "category", "sub_category"]
for c in text_cols:
    df[c] = df[c].astype(str).str.strip()

df[text_cols].head(3)

,ship_mode,customer_name,segment,country,city,state,region,product_name,category,sub_category
0,Second Class,Claire Gute,Consumer,United States,Henderson,Kentucky,South,Bush Somerset Collection Bookcase,Furniture,Bookcases
1,Second Class,Claire Gute,Consumer,United States,Henderson,Kentucky,South,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs
2,Second Class,Darrin Van Huff,Corporate,United States,Los Angeles,California,West,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels


Quick checks

In [ ]:
print(df["category"].value_counts())
print(df["region"].value_counts())
print(df["ship_mode"].value_counts())

category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64
region
West       3203
East       2848
Central    2323
South      1620
Name: count, dtype: int64
ship_mode
Standard Class    5968
Second Class      1945
First Class       1538
Same Day           543
Name: count, dtype: int64


Validate numeric ranges

In [ ]:
# discount should be between 0 and 1
print(df[(df["discount"] < 0 ) | (df["discount"] > 1 )].shape)

# quantity should be >= 1
print(df[df["quantity"] <= 0].shape)

# sales typically > 0 (profit can be negative)
print(df[df["sales"] <= 0 ].shape)

(0, 21)
(0, 21)
(0, 21)


Creating helper columns

In [ ]:
df["order_year"] = df["order_date"].dt.year
df["order_month"] = df["order_date"].dt.to_period("M").astype(str)

df[["order_year", "order_month"]]

,order_year,order_month
0,2016,2016-11
1,2016,2016-11
2,2016,2016-06
3,2015,2015-10
4,2015,2015-10
...,...,...
9989,2014,2014-01
9990,2017,2017-02
9991,2017,2017-02
9992,2017,2017-02


Final Sanity KPIs

In [ ]:
sanity = { 
    "rows": len(df),
    "orders": df["order_id"].nunique(),
    "customers": df["customer_id"].nunique(),
    "revenue" : float(df["sales"].sum()),
    "profit" : float(df["profit"].sum())
}
sanity

{'rows': 9994,
 'orders': 5009,
 'customers': 793,
 'revenue': 2297200.8603,
 'profit': 286397.0217}

Saving the sanity KPIs

In [ ]:
with open("E:\\Data Analysis\\Projects\\Data Analysis End to end\\Data\\data_quality_report.txt", "w") as f:
    for k,v in sanity.items():
        f.write(f"{k}: {v}\n")

Exporting cleaned dataset

In [ ]:
df_clean = df.copy()

from sqlalchemy import create_engine
user = "root"
password = "0301"
host = "localhost"
port = 3306
db = "superstore"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}")

In [ ]:
# This will create a table named stg_superstore_clean and load data
df_clean.to_sql(
    name = "stg_superstore_clean",
    con = engine,
    if_exists = "replace",
    index = False,
    chunksize = 10000
)
print("Loaded: ", len(df_clean), "rows")

Loaded:  9994 rows
